In [ ]:
import os
import sys
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from datetime import datetime
from pathlib import Path
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from matplotlib.patches import Rectangle

import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.util import add_cyclic_point
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import warnings

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
# set fonts configuration - Arial
matplotlib.rcParams['font.family'] = "Open Sans"
matplotlib.rcParams['font.sans-serif'] = "Arial"

In [ ]:
era5_env_dir = Path('/neelin2020/mcs_flextrkr/era5_envs/')
featstats_dir = Path('/scratch/wmtsai/temp_mcs/mcs_stats/mcs_tracks_non2mcs/tracks_area_mean/')
mcsstats_dir = Path('/scratch/wmtsai/temp_mcs/mcs_stats/mcs_tracks_non2mcs')

In [ ]:
def BL2dhist_var_condition_ERA5env(var_name, year_list):
    
    """
    get conditional average of the variable binned by BL,cape and BL,subsat
    """

    # bins for BL_CAPE and BL_SUBSAT
    bins_cape = np.linspace(-10,10,101)
    bins_subsat = np.linspace(-5,15,101)
    dbl_cape = (bins_cape[1]-bins_cape[0])/2
    dbl_subsat= (bins_subsat[1]-bins_subsat[0])/2

    samples = np.zeros((6, 2, len(bins_cape)-1, len(bins_subsat)-1)) # (mcs_phase, surface_type, BL-cape, BL-subsat) 
    var_sum = np.zeros((6, 2, len(bins_cape)-1, len(bins_subsat)-1))
    var_num = np.copy(var_sum) 
    
    print('processing variable name in era5-env data: {}'.format(var_name))

    for year in year_list:

        print('processing year: {}'.format(year))

        data_era5_env = xr.open_dataset(era5_env_dir / 'mcs_era5_mean_envs_{}0101.0000_{}0101.0000.nc'.format(year,year+1))
        data_featstats = xr.open_dataset(featstats_dir / 'featstats_tracks_non2mcs_{}.tropics30NS.extend.nc'.format(year))
        data_mcsstats = xr.open_dataset(mcsstats_dir / 'mcs_tracks_non2mcs_{}.tropics30NS.extend.nc'.format(year))
        
        # add extra variable
        data_era5_env['w_200mb'] = data_era5_env.W.sel(level='200',method='nearest')
        
        # extract MCSs over ocean and land
        idt_mcs_init = data_mcsstats.idt_mcs_init
        ls_flag = data_mcsstats.landsea_flag.sel(times=idt_mcs_init) # MCS surface type at Init.
        idx_land = np.where(ls_flag == 1)[0]
        idx_ocean = np.where(ls_flag == 0)[0]
        
        # extract the corresponding non2MCS tracks from the original tracks
        tracks_land = data_mcsstats.tracks.isel(tracks=idx_land) # real track tag
        tracks_ocean = data_mcsstats.tracks.isel(tracks=idx_ocean) 
        
        for n, tracks_sel in enumerate([tracks_ocean, tracks_land]):
        
            data_mcsstats_sub = data_mcsstats.sel(tracks=tracks_sel)
            data_featstats_sub = data_featstats.sel(tracks=tracks_sel)
            data_era5_env_sub = data_era5_env.sel(tracks=tracks_sel)

            for p,(phase,var_string) in enumerate(zip(['CCS','Init','Grow','Mature','Decay','End']
                                                      ,['idt_ccs_init','idt_mcs_init','idt_mcs_grow',
                                                        'idt_mcs_mature','idt_mcs_decay','idt_mcs_end'])):

                BL_CAPE_amean = data_featstats_sub.BL_CAPE_3degmean.sel(mcs_phase=phase).values
                BL_SUBSAT_amean = data_featstats_sub.BL_SUBSAT_3degmean.sel(mcs_phase=phase).values

                for i in range(len(bins_cape)-1):
                    idx = np.where(np.logical_and(BL_CAPE_amean >= bins_cape[i], BL_CAPE_amean < bins_cape[i+1]))[0]
                    for j in range(len(bins_subsat)-1):
                        idy = np.where(np.logical_and(BL_SUBSAT_amean >= bins_subsat[j], BL_SUBSAT_amean < bins_subsat[j+1]))[0]
                        idx_com = np.intersect1d(idx,idy)

                        samples[p,n,i,j] += len(idx_com)

                        if len(idx_com) > 0: # if getting samples                    
                            # get shear info at the corresponding times
                            idt_phase = data_mcsstats_sub[var_string].isel(tracks=idx_com)                   
                            for idc,idt in zip(idx_com, idt_phase):                    
                                if ~np.isnan(data_era5_env_sub[var_name].isel(tracks=idc).sel(rel_times=idt)) == 1:
                                    var_sum[p,n,i,j] += data_era5_env_sub[var_name].isel(tracks=idc).sel(rel_times=idt)
                                    var_num[p,n,i,j] += 1

    # write into xarray dataset
    ds = xr.Dataset(data_vars=dict(samples=(['mcs_phase','surface_type','bins_cape','bins_subsat'],samples),
                                   var_sum=(['mcs_phase','surface_type','bins_cape','bins_subsat'],var_sum),
                                   var_num=(['mcs_phase','surface_type','bins_cape','bins_subsat'],var_num)),
                    coords=dict(mcs_phase=(['mcs_phase'],['CCS','Init','Grow','Mature','Decay','End']),
                                surface_type=(['surface_type'],['ocean','land']),
                                bins_cape=(['bins_cape'],bins_cape[:-1]),
                                bins_subsat=(['bins_subsat'],bins_subsat[:-1])))    
    
    return ds

In [ ]:
def BL2dhist_var_condition_MCSstats(var_name, year_list, single_value=False):
    
    """
    get conditional average of the variable binned by BL,cape and BL,subsat
    """

    # bins for BL_CAPE and BL_SUBSAT
    bins_cape = np.linspace(-10,10,101)
    bins_subsat = np.linspace(-5,15,101)
    dbl_cape = (bins_cape[1]-bins_cape[0])/2
    dbl_subsat= (bins_subsat[1]-bins_subsat[0])/2

    samples = np.zeros((6, 2, len(bins_cape)-1, len(bins_subsat)-1)) # (mcs_phase, surface_type, BL-cape, BL-subsat) 
    var_sum = np.zeros((6, 2, len(bins_cape)-1, len(bins_subsat)-1))
    var_num = np.copy(var_sum) 
    
    print('processing variable name in mcsstats data: {}'.format(var_name))

    for year in year_list:

        print('processing year: {}'.format(year))

        data_era5_env = xr.open_dataset(era5_env_dir / 'mcs_era5_mean_envs_{}0101.0000_{}0101.0000.nc'.format(year,year+1))
        data_featstats = xr.open_dataset(featstats_dir / 'featstats_tracks_non2mcs_{}.tropics30NS.extend.nc'.format(year))
        data_mcsstats = xr.open_dataset(mcsstats_dir / 'mcs_tracks_non2mcs_{}.tropics30NS.extend.nc'.format(year))
        
        # adding post-defined variables
        data_mcsstats['hour_mature2end'] = data_mcsstats['idt_mcs_end'] - data_mcsstats['idt_mcs_mature'] 
        
        # extract MCSs over ocean and land
        idt_mcs_init = data_mcsstats.idt_mcs_init
        ls_flag = data_mcsstats.landsea_flag.sel(times=idt_mcs_init) # MCS surface type at Init.
        idx_land = np.where(ls_flag == 1)[0]
        idx_ocean = np.where(ls_flag == 0)[0]
        
        # extract the corresponding non2MCS tracks from the original tracks
        tracks_land = data_mcsstats.tracks.isel(tracks=idx_land)
        tracks_ocean = data_mcsstats.tracks.isel(tracks=idx_ocean)
        
        for n, tracks_sel in enumerate([tracks_ocean, tracks_land]):
            
            data_mcsstats_sub = data_mcsstats.sel(tracks=tracks_sel)
            data_featstats_sub = data_featstats.sel(tracks=tracks_sel)

            for p,(phase,var_string) in enumerate(zip(['CCS','Init','Grow','Mature','Decay','End']
                                                      ,['idt_ccs_init','idt_mcs_init','idt_mcs_grow',
                                                        'idt_mcs_mature','idt_mcs_decay','idt_mcs_end'])):

                BL_CAPE_amean = data_featstats_sub.BL_CAPE_3degmean.sel(mcs_phase=phase).values
                BL_SUBSAT_amean = data_featstats_sub.BL_SUBSAT_3degmean.sel(mcs_phase=phase).values

                for i in range(len(bins_cape)-1):
                    idx = np.where(np.logical_and(BL_CAPE_amean >= bins_cape[i], BL_CAPE_amean < bins_cape[i+1]))[0]
                    for j in range(len(bins_subsat)-1):
                        idy = np.where(np.logical_and(BL_SUBSAT_amean >= bins_subsat[j], BL_SUBSAT_amean < bins_subsat[j+1]))[0]
                        idx_com = np.intersect1d(idx,idy)

                        samples[p,n,i,j] += len(idx_com)

                        if len(idx_com) > 0: # if getting samples                    
                            # get shear info at the corresponding times
                            idt_phase = data_mcsstats_sub[var_string].isel(tracks=idx_com)  

                            if single_value == False: # var[times]

                                for idc,idt in zip(idx_com, idt_phase):  
                                    var_sum[p,n,i,j] += data_mcsstats_sub[var_name].isel(tracks=idc).sel(times=idt)
                                    var_num[p,n,i,j] += 1

                            else: # var is a single value for each MCS track e.g., mcs_duration
                                for idc,idt in zip(idx_com, idt_phase):                    
                                    var_sum[p,n,i,j] += data_mcsstats_sub[var_name].isel(tracks=idc)
                                    var_num[p,n,i,j] += 1

    # write into xarray dataset
    ds = xr.Dataset(data_vars=dict(samples=(['mcs_phase','surface_type','bins_cape','bins_subsat'],samples),
                                   var_sum=(['mcs_phase','surface_type','bins_cape','bins_subsat'],var_sum),
                                   var_num=(['mcs_phase','surface_type','bins_cape','bins_subsat'],var_num)),
                    coords=dict(mcs_phase=(['mcs_phase'],['CCS','Init','Grow','Mature','Decay','End']),
                                surface_type=(['surface_type'],['ocean','land']),
                                bins_cape=(['bins_cape'],bins_cape[:-1]),
                                bins_subsat=(['bins_subsat'],bins_subsat[:-1])))    
    
    return ds

In [ ]:
%%time
ds = BL2dhist_var_condition_ERA5env(var_name='w_500mb', year_list=np.arange(2001,2011))

In [ ]:
fig,((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3,figsize=(10,5))

var_name = 'w_500mb'
surface_type = 'ocean'

for ax,mcs_phase in zip([ax1,ax2,ax3,ax4,ax5,ax6],['CCS','Init','Grow','Mature','Decay','End']):
    
    samples = ds.sel(mcs_phase=mcs_phase, surface_type=surface_type).samples
    var_sum = ds.sel(mcs_phase=mcs_phase, surface_type=surface_type).var_sum
    var_num = ds.sel(mcs_phase=mcs_phase, surface_type=surface_type).var_num
    var_num = var_num.where(var_num > 10)
    
    # smoothening samples
    var_mean = var_sum/var_num
    dbl = 0.2
    var_mean_interp = var_mean.interp(bins_cape=np.arange(ds.bins_cape[0],ds.bins_cape[-1]+dbl,dbl),
                          bins_subsat=np.arange(ds.bins_subsat[0],ds.bins_subsat[-1]+dbl,dbl))
    samples_interp = samples.interp(bins_cape=np.arange(ds.bins_cape[0],ds.bins_cape[-1]+dbl,dbl),
                          bins_subsat=np.arange(ds.bins_subsat[0],ds.bins_subsat[-1]+dbl,dbl))
    
    cp = ax.pcolormesh(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, var_mean_interp
                       ,vmin=-0.5, vmax=0.5, cmap='RdBu', zorder=2)
    ax.contour(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, var_mean_interp, levels=[0],colors=['k'],
              linewidths=0.5,zorder=3)
    cbar = plt.colorbar(cp, ax=ax, shrink=0.7)
    cbar.set_label('velocity (Pa/s)',fontsize=10)
    
    # add pdf of samples 
    #ax.contour(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, samples_interp/samples_interp.sum()
    #           , levels=[0.01,0.02,0.03,0.04,0.05], colors=['grey'],linewidths=1,zorder=3)    

    # referecne line of BL = 0K
    ax.plot(np.linspace(-2,10,13),np.linspace(-2,10,13), color='k', lw=1)
    
    ax.set_title(mcs_phase, fontsize=11)
    ax.set_xlabel('$\overline{B}_{L,subsat,mcs}$ (K)',fontsize=10)
    ax.set_ylabel('$\overline{B}_{L,cape,mcs}$ (K)',fontsize=10)
    ax.set_yticks([-2,0,2,4,6,8,10])
    ax.set_xticks([0,2,4,6,8,10])
    ax.set_ylim([-2,4])
    ax.set_xlim([0,9])
    ax.grid(linestyle=':',lw=1,color='grey',zorder=1)
    
plt.suptitle('Variable: {}, {}'.format(var_name, surface_type),fontsize=10, fontweight='bold',y=1)
plt.tight_layout()

In [ ]:
fig,((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3,figsize=(10,5))

var_name = 'w_500mb'
surface_type = 'land'

for ax,mcs_phase in zip([ax1,ax2,ax3,ax4,ax5,ax6],['CCS','Init','Grow','Mature','Decay','End']):
    
    samples = ds.sel(mcs_phase=mcs_phase, surface_type=surface_type).samples
    var_sum = ds.sel(mcs_phase=mcs_phase, surface_type=surface_type).var_sum
    var_num = ds.sel(mcs_phase=mcs_phase, surface_type=surface_type).var_num
    
    # smoothening samples
    var_mean = var_sum/var_num
    dbl = 0.2
    var_mean_interp = var_mean.interp(bins_cape=np.arange(ds.bins_cape[0],ds.bins_cape[-1]+dbl,dbl),
                          bins_subsat=np.arange(ds.bins_subsat[0],ds.bins_subsat[-1]+dbl,dbl))
    samples_interp = samples.interp(bins_cape=np.arange(ds.bins_cape[0],ds.bins_cape[-1]+dbl,dbl),
                          bins_subsat=np.arange(ds.bins_subsat[0],ds.bins_subsat[-1]+dbl,dbl))
    
    cp = ax.pcolormesh(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, var_mean_interp
                       ,vmin=-0.4, vmax=0.4, cmap='RdBu', zorder=2)
    ax.contour(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, var_mean_interp, levels=[0],colors=['k'],
              linewidths=0.5,zorder=3)
    cbar = plt.colorbar(cp, ax=ax, shrink=0.7)
    cbar.set_label('velocity (Pa/s)',fontsize=10)
    
    # add pdf of samples 
    #ax.contour(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, samples_interp/samples_interp.sum()
    #           , levels=[0.01,0.02,0.03,0.04,0.05], colors=['grey'],linewidths=1,zorder=3)    

    # referecne line of BL = 0K
    ax.plot(np.linspace(-2,10,13),np.linspace(-2,10,13), color='k', lw=1)
    
    ax.set_title(mcs_phase, fontsize=11)
    ax.set_xlabel('$\overline{B}_{L,subsat,mcs}$ (K)',fontsize=10)
    ax.set_ylabel('$\overline{B}_{L,cape,mcs}$ (K)',fontsize=10)
    ax.set_yticks([-2,0,2,4,6,8,10])
    ax.set_xticks([0,2,4,6,8,10])
    ax.set_ylim([-2,4])
    ax.set_xlim([0,9])
    ax.grid(linestyle=':',lw=1,color='grey',zorder=1)
    
plt.suptitle('Variable: {}, {}'.format(var_name, surface_type),fontsize=10, fontweight='bold',y=1)
plt.tight_layout()

In [ ]:
%%time
ds_850 = BL2dhist_var_condition_ERA5env(var_name='w_850mb', year_list=np.arange(2001,2011))

In [ ]:
fig,((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3,figsize=(10,5))

var_name = 'w_850mb'
surface_type = 'ocean'

for ax,mcs_phase in zip([ax1,ax2,ax3,ax4,ax5,ax6],['CCS','Init','Grow','Mature','Decay','End']):
    
    samples = ds_850.sel(mcs_phase=mcs_phase, surface_type=surface_type).samples
    var_sum = ds_850.sel(mcs_phase=mcs_phase, surface_type=surface_type).var_sum
    var_num = ds_850.sel(mcs_phase=mcs_phase, surface_type=surface_type).var_num
    
    # smoothening samples
    var_mean = var_sum/var_num
    dbl = 0.2
    var_mean_interp = var_mean.interp(bins_cape=np.arange(var_sum.bins_cape[0],var_sum.bins_cape[-1]+dbl,dbl),
                          bins_subsat=np.arange(var_sum.bins_subsat[0],var_sum.bins_subsat[-1]+dbl,dbl))
    samples_interp = samples.interp(bins_cape=np.arange(var_sum.bins_cape[0],var_sum.bins_cape[-1]+dbl,dbl),
                          bins_subsat=np.arange(var_sum.bins_subsat[0],var_sum.bins_subsat[-1]+dbl,dbl))
    
    cp = ax.pcolormesh(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, var_mean_interp
                       ,vmin=-0.4, vmax=0.4, cmap='RdBu', zorder=2)
    ax.contour(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, var_mean_interp, levels=[0],colors=['k'],
              linewidths=0.5,zorder=3)
    cbar = plt.colorbar(cp, ax=ax, shrink=0.7)
    cbar.set_label('velocity (Pa/s)',fontsize=10)
    
    # add pdf of samples 
    ax.contour(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, samples_interp/samples_interp.sum()
               , levels=[0.01,0.02,0.03,0.04,0.05], colors=['grey'],linewidths=1,zorder=3)    

    # referecne line of BL = 0K
    ax.plot(np.linspace(-2,10,13),np.linspace(-2,10,13), color='k', lw=1)
    
    ax.set_xlabel('$\overline{B}_{L,subsat,3deg}$ (K)',fontsize=10)
    ax.set_ylabel('$\overline{B}_{L,cape,3deg}$ (K)',fontsize=10)
    #ax.set_title('MCS duration, uncondtional on shear \n {}, (samples > 30)'.format(surface_type), fontsize=9.5)
    ax.set_yticks([-2,0,2,4,6,8,10])
    ax.set_xticks([0,2,4,6,8,10])
    ax.set_ylim([-2,4])
    ax.set_xlim([0,9])
    ax.grid(linestyle=':',lw=1,color='grey',zorder=1)
    
plt.suptitle('Variable: {}, {}'.format(var_name, surface_type),fontsize=10, fontweight='bold',y=1)
plt.tight_layout()

In [ ]:
fig,((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3,figsize=(10,5))

var_name = 'w_850mb'
surface_type = 'land'

for ax,mcs_phase in zip([ax1,ax2,ax3,ax4,ax5,ax6],['CCS','Init','Grow','Mature','Decay','End']):
    
    samples = ds_850.sel(mcs_phase=mcs_phase, surface_type=surface_type).samples
    var_sum = ds_850.sel(mcs_phase=mcs_phase, surface_type=surface_type).var_sum
    var_num = ds_850.sel(mcs_phase=mcs_phase, surface_type=surface_type).var_num
    
    # smoothening samples
    var_mean = var_sum/var_num
    dbl = 0.2
    var_mean_interp = var_mean.interp(bins_cape=np.arange(var_sum.bins_cape[0],var_sum.bins_cape[-1]+dbl,dbl),
                          bins_subsat=np.arange(var_sum.bins_subsat[0],var_sum.bins_subsat[-1]+dbl,dbl))
    samples_interp = samples.interp(bins_cape=np.arange(var_sum.bins_cape[0],var_sum.bins_cape[-1]+dbl,dbl),
                          bins_subsat=np.arange(var_sum.bins_subsat[0],var_sum.bins_subsat[-1]+dbl,dbl))
    
    cp = ax.pcolormesh(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, var_mean_interp
                       ,vmin=-0.4, vmax=0.4, cmap='RdBu', zorder=2)
    ax.contour(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, var_mean_interp, levels=[0],colors=['k'],
              linewidths=0.5,zorder=3)
    cbar = plt.colorbar(cp, ax=ax, shrink=0.7)
    cbar.set_label('velocity (Pa/s)',fontsize=10)
    
    # add pdf of samples 
    ax.contour(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, samples_interp/samples_interp.sum()
               , levels=[0.01,0.02,0.03,0.04,0.05], colors=['grey'],linewidths=1,zorder=3)    

    # referecne line of BL = 0K
    ax.plot(np.linspace(-2,10,13),np.linspace(-2,10,13), color='k', lw=1)
    
    ax.set_xlabel('$\overline{B}_{L,subsat,3deg}$ (K)',fontsize=10)
    ax.set_ylabel('$\overline{B}_{L,cape,3deg}$ (K)',fontsize=10)
    #ax.set_title('MCS duration, uncondtional on shear \n {}, (samples > 30)'.format(surface_type), fontsize=9.5)
    ax.set_yticks([-2,0,2,4,6,8,10])
    ax.set_xticks([0,2,4,6,8,10])
    ax.set_ylim([-2,4])
    ax.set_xlim([0,9])
    ax.grid(linestyle=':',lw=1,color='grey',zorder=1)
    
plt.suptitle('Variable: {}, {}'.format(var_name, surface_type),fontsize=10, fontweight='bold',y=1)
plt.tight_layout()

In [ ]:
%%time
ds = BL2dhist_var_condition_ERA5env(var_name='w_200mb', year_list=np.arange(2001,2011))

#### BL2Dhist conditional average of MCS stats 

In [ ]:
%%time
ds_duration = BL2dhist_var_condition_MCSstats(var_name='mcs_duration', year_list=np.arange(2001,2021), single_value=True)

In [ ]:
fig,((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3,figsize=(10,5))

var_name = 'mcs_duration'

for ax,mcs_phase in zip([ax1,ax2,ax3,ax4,ax5,ax6],['CCS','Init','Grow','Mature','Decay','End']):
    
    samples = ds_duration.sel(mcs_phase=mcs_phase).samples.sum('surface_type')
    var_sum = ds_duration.sel(mcs_phase=mcs_phase).var_sum.sum('surface_type')
    var_num = ds_duration.sel(mcs_phase=mcs_phase).var_num.sum('surface_type')
    
    var_num = var_num.where(var_num > 20)
    var_sum = var_sum.where(var_num > 20)
    
    # smoothening samples
    var_mean = var_sum/var_num
    dbl = 0.5
    var_mean_interp = var_mean.interp(bins_cape=np.arange(var_sum.bins_cape[0],var_sum.bins_cape[-1]+dbl,dbl),
                          bins_subsat=np.arange(var_sum.bins_subsat[0],var_sum.bins_subsat[-1]+dbl,dbl))
    samples_interp = samples.interp(bins_cape=np.arange(var_sum.bins_cape[0],var_sum.bins_cape[-1]+dbl,dbl),
                          bins_subsat=np.arange(var_sum.bins_subsat[0],var_sum.bins_subsat[-1]+dbl,dbl))
    
    cp = ax.pcolormesh(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, var_mean_interp
                       ,vmin=5, vmax=20, cmap='jet', zorder=2)
    ax.contour(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, var_mean_interp, levels=[0,15,17,19],colors=['k'],
              linewidths=0.5,zorder=3)
    cbar = plt.colorbar(cp, ax=ax, shrink=0.7, extend='max')
    cbar.set_label('MCS duration (hr)',fontsize=10)
    
    # add pdf of samples 
    #ax.contour(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, samples_interp/samples_interp.sum()
    #           , levels=np.arange(0.0025,0.02,0.0025), colors=['grey'],linewidths=1,zorder=3)    

    # referecne line of BL = 0K
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13), color='k', lw=1.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-2, color='grey', ls='-',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-4, color='grey', ls='-',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-6, color='grey', ls='-',lw=0.5)
    
    ax.set_xlabel('$\overline{B}_{L,subsat}$ (K)',fontsize=10)
    ax.set_ylabel('$\overline{B}_{L,cape}$ (K)',fontsize=10)
    ax.set_title(mcs_phase, fontsize=11)

    ax.set_yticks([-4,-2,0,2,4,6,8,10,12])
    ax.set_xticks([-2,0,2,4,6,8,10,12])
    ax.grid(linestyle=':',lw=1,color='grey',zorder=1)
    
    # set x-y lims
    ax.set_ylim([-4,8])
    ax.set_xlim([0,14])
    
plt.suptitle('Variable: {}'.format(var_name),fontsize=10, fontweight='bold',y=1)
plt.tight_layout()

In [ ]:
%%time
ds_hr_mat2end = BL2dhist_var_condition_MCSstats(var_name='hour_mature2end', year_list=np.arange(2001,2021), single_value=True)

In [ ]:
fig,((ax1,ax2)) = plt.subplots(1,2,figsize=(6,3.2))

var_name = 'Lasting time between Mature and End (hr)'

for ax,mcs_phase,surface_type,title_label in zip([ax1,ax2],['Mature','Mature'],['ocean','land'],['(a)','(b)']):
    
    samples = ds_hr_mat2end.sel(mcs_phase=mcs_phase,surface_type=surface_type).samples
    var_sum = ds_hr_mat2end.sel(mcs_phase=mcs_phase,surface_type=surface_type).var_sum
    var_num = ds_hr_mat2end.sel(mcs_phase=mcs_phase,surface_type=surface_type).var_num
    
    dbl = 0.25
    var_num_interp = var_num.interp(bins_cape=np.arange(var_sum.bins_cape[0],var_sum.bins_cape[-1]+dbl,dbl),
                          bins_subsat=np.arange(var_sum.bins_subsat[0],var_sum.bins_subsat[-1]+dbl,dbl))
    var_sum_interp = var_sum.interp(bins_cape=np.arange(var_sum.bins_cape[0],var_sum.bins_cape[-1]+dbl,dbl),
                          bins_subsat=np.arange(var_sum.bins_subsat[0],var_sum.bins_subsat[-1]+dbl,dbl))
    
    var_num_interp = var_num_interp.where(var_num_interp > 100)
    var_sum_interp = var_sum_interp.where(var_num_interp > 100)
    var_mean_interp = var_sum_interp/var_num_interp
    
    cp = ax.pcolormesh(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, var_mean_interp
                       ,vmin=5, vmax=10, cmap='YlGnBu', zorder=2)
    cf = ax.contour(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, var_mean_interp, levels=[0,7,8,9,10,11]
                ,colors=['violet'],linewidths=1.2,zorder=2)
    
    if surface_type == 'land':
        cbaxes = fig.add_axes([0.99, 0.295, 0.01, 0.4])              
        cbar = plt.colorbar(cp, cax=cbaxes, shrink=0.7)
        cbar.set_label('Lasting time (hr)',fontsize=9)
        cbar.set_ticks([5,6,7,8,9,10]) 

    # referecne line of BL = 0K
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13), color='k', lw=1.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-2, color='k', ls='-',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-4, color='k', ls='-',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-6, color='k', ls='-',lw=0.5)
    
    ax.set_xlabel('$\overline{B}_{L,subsat}$ (K)',fontsize=10.5, labelpad=0)
    ax.set_ylabel('$\overline{B}_{L,cape}$ (K)',fontsize=10.5, labelpad=0)
    ax.set_title('{} MCS {} phase, {}'.format(title_label, mcs_phase, surface_type), fontsize=10.5)

    ax.set_yticks([-4,-2,0,2,4,6,8,10,12])
    ax.set_xticks([-2,0,2,4,6,8,10,12])
    ax.grid(linestyle=':',lw=0.5,color='grey',zorder=1)
    
    # set x-y lims
    ax.set_ylim([-2,6])
    ax.set_xlim([0,10])
    
plt.suptitle('Conditional average of lasting time (hr)',fontsize=10, fontweight='bold',y=1)
plt.tight_layout()

#fig.savefig('/scratch/wmtsai/temp_mcs/fig/BLhist2d_condi_lastingtime_ocean_land.png',dpi=300
#           ,bbox_inches='tight', transparent=False)

In [ ]:
fig,((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3,figsize=(10,5))

var_name = 'mcs_duration'
surface_type = 'ocean'

for ax,mcs_phase in zip([ax1,ax2,ax3,ax4,ax5,ax6],['CCS','Init','Grow','Mature','Decay','End']):
    
    samples = ds_duration.sel(mcs_phase=mcs_phase, surface_type='ocean').samples
    var_sum = ds_duration.sel(mcs_phase=mcs_phase, surface_type='ocean').var_sum
    var_num = ds_duration.sel(mcs_phase=mcs_phase, surface_type='ocean').var_num
    
    # smoothening samples
    var_mean = var_sum/var_num
    dbl = 0.75
    var_mean_interp = var_mean.interp(bins_cape=np.arange(var_sum.bins_cape[0],var_sum.bins_cape[-1]+dbl,dbl),
                          bins_subsat=np.arange(var_sum.bins_subsat[0],var_sum.bins_subsat[-1]+dbl,dbl))
    samples_interp = samples.interp(bins_cape=np.arange(var_sum.bins_cape[0],var_sum.bins_cape[-1]+dbl,dbl),
                          bins_subsat=np.arange(var_sum.bins_subsat[0],var_sum.bins_subsat[-1]+dbl,dbl))
    
    cp = ax.pcolormesh(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, var_mean_interp
                       ,vmin=5, vmax=20, cmap='jet', zorder=2)
    ax.contour(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, var_mean_interp, levels=[0,15],colors=['k'],
              linewidths=0.5,zorder=3)
    cbar = plt.colorbar(cp, ax=ax, shrink=0.7, extend='max')
    cbar.set_label('MCS duration (hr)',fontsize=10)
    
    # add pdf of samples 
    #ax.contour(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, samples_interp/samples_interp.sum()
    #           , levels=np.arange(0.0025,0.02,0.0025), colors=['grey'],linewidths=1,zorder=3)    

    # referecne line of BL = 0K
    ax.plot(np.linspace(-2,10,13),np.linspace(-2,10,13), color='k', lw=1)
    
    ax.set_xlabel('$\hat{B}_{L,subsat}$ (K)',fontsize=10)
    ax.set_ylabel('$\hat{B}_{L,cape}$ (K)',fontsize=10)
    ax.set_title(mcs_phase, fontsize=11)
    ax.set_ylim([-2,10])
    ax.set_xlim([-2,12])
    ax.set_yticks([0,2,4,6,8,10])
    ax.set_xticks([0,2,4,6,8,10])
    ax.grid(linestyle=':',lw=1,color='grey',zorder=1)
    
plt.suptitle('Variable: {}, {}'.format(var_name, surface_type),fontsize=10, fontweight='bold',y=1)
plt.tight_layout()

In [ ]:
fig,((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3,figsize=(10,5))

var_name = 'mcs_duration'
surface_type = 'land'

for ax,mcs_phase in zip([ax1,ax2,ax3,ax4,ax5,ax6],['CCS','Init','Grow','Mature','Decay','End']):
    
    samples = ds_duration.sel(mcs_phase=mcs_phase, surface_type='ocean').samples
    var_sum = ds_duration.sel(mcs_phase=mcs_phase, surface_type='ocean').var_sum
    var_num = ds_duration.sel(mcs_phase=mcs_phase, surface_type='ocean').var_num
    
    # smoothening samples
    var_mean = var_sum/var_num
    dbl = 0.75
    var_mean_interp = var_mean.interp(bins_cape=np.arange(var_sum.bins_cape[0],var_sum.bins_cape[-1]+dbl,dbl),
                          bins_subsat=np.arange(var_sum.bins_subsat[0],var_sum.bins_subsat[-1]+dbl,dbl))
    samples_interp = samples.interp(bins_cape=np.arange(var_sum.bins_cape[0],var_sum.bins_cape[-1]+dbl,dbl),
                          bins_subsat=np.arange(var_sum.bins_subsat[0],var_sum.bins_subsat[-1]+dbl,dbl))
    
    cp = ax.pcolormesh(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, var_mean_interp
                       ,vmin=5, vmax=20, cmap='jet', zorder=2)
    ax.contour(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, var_mean_interp, levels=[0,15],colors=['k'],
              linewidths=0.5,zorder=3)
    cbar = plt.colorbar(cp, ax=ax, shrink=0.7, extend='max')
    cbar.set_label('MCS duration (hr)',fontsize=10)
    
    # add pdf of samples 
    #ax.contour(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, samples_interp/samples_interp.sum()
    #           , levels=np.arange(0.0025,0.02,0.0025), colors=['grey'],linewidths=1,zorder=3)    

    # referecne line of BL = 0K
    ax.plot(np.linspace(-2,10,13),np.linspace(-2,10,13), color='k', lw=1)
    
    ax.set_xlabel('$\hat{B}_{L,subsat}$ (K)',fontsize=10)
    ax.set_ylabel('$\hat{B}_{L,cape}$ (K)',fontsize=10)
    ax.set_title(mcs_phase, fontsize=11)
    ax.set_ylim([-2,10])
    ax.set_xlim([-2,12])
    ax.set_yticks([0,2,4,6,8,10])
    ax.set_xticks([0,2,4,6,8,10])
    ax.grid(linestyle=':',lw=1,color='grey',zorder=1)
    
plt.suptitle('Variable: {}, {}'.format(var_name, surface_type),fontsize=10, fontweight='bold',y=1)
plt.tight_layout()

#### other available variables 

In [ ]:
%%time
ds_MUCIN = BL2dhist_var_condition(var_name='MUCIN', year_list=np.arange(2001,2011))

In [ ]:
fig,((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3,figsize=(10,5))

var_name = 'MUCIN'

for ax,mcs_phase in zip([ax1,ax2,ax3,ax4,ax5,ax6],['CCS','Init','Grow','Mature','Decay','End']):
    
    samples = ds_MUCIN.sel(mcs_phase=mcs_phase).samples
    var_sum = ds_MUCIN.sel(mcs_phase=mcs_phase).var_sum
    var_num = ds_MUCIN.sel(mcs_phase=mcs_phase).var_num
    
    # smoothening samples
    var_mean = var_sum/var_num
    dbl = 1
    var_mean_interp = var_mean.interp(bins_cape=np.arange(var_sum.bins_cape[0],var_sum.bins_cape[-1]+dbl,dbl),
                          bins_subsat=np.arange(var_sum.bins_subsat[0],var_sum.bins_subsat[-1]+dbl,dbl))
    samples_interp = samples.interp(bins_cape=np.arange(var_sum.bins_cape[0],var_sum.bins_cape[-1]+dbl,dbl),
                          bins_subsat=np.arange(var_sum.bins_subsat[0],var_sum.bins_subsat[-1]+dbl,dbl))
    
    cp = ax.pcolormesh(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, var_mean_interp
                       , vmin=0, vmax=-50, cmap='jet', zorder=2)
    #ax.contour(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, var_mean_interp, levels=5,colors=['k'],
    #          linewidths=0.5,zorder=3)
    cbar = plt.colorbar(cp, ax=ax, shrink=0.7)
    cbar.set_label('MUCIN (J/kg)',fontsize=10)
    
    # add pdf of samples 
    ax.contour(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, samples_interp/samples_interp.sum()
               , levels=[0.01,0.02,0.03,0.04,0.05], colors=['grey'],linewidths=1,zorder=3)    

    # referecne line of BL = 0K
    ax.plot(np.linspace(-2,10,13),np.linspace(-2,10,13), color='k', lw=1)
    
    ax.set_xlabel('B$_{L,subsat}$ (K)',fontsize=10)
    ax.set_ylabel('B$_{L,cape}$ (K)',fontsize=10)
    ax.set_title(mcs_phase, fontsize=11)
    ax.set_ylim([-2,10])
    ax.set_xlim([-2,12])
    ax.set_yticks([0,2,4,6,8,10])
    ax.set_xticks([0,2,4,6,8,10])
    ax.grid(linestyle=':',lw=1,color='grey',zorder=1)
    
plt.suptitle('Variable: {}'.format(var_name),fontsize=10, fontweight='bold',y=1)
plt.tight_layout()

In [ ]:
%%time
ds_VIWVD = BL2dhist_var_condition(var_name='VIWVD', year_list=np.arange(2001,2011))

In [ ]:
fig,((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3,figsize=(10,5))

var_name = 'VIWVD'

for ax,mcs_phase in zip([ax1,ax2,ax3,ax4,ax5,ax6],['CCS','Init','Grow','Mature','Decay','End']):
    
    samples = ds_VIWVD.sel(mcs_phase=mcs_phase).samples
    var_sum = ds_VIWVD.sel(mcs_phase=mcs_phase).var_sum
    var_num = ds_VIWVD.sel(mcs_phase=mcs_phase).var_num
    
    # smoothening samples
    var_mean = var_sum/var_num
    dbl = 1
    var_mean_interp = var_mean.interp(bins_cape=np.arange(var_sum.bins_cape[0],var_sum.bins_cape[-1]+dbl,dbl),
                          bins_subsat=np.arange(var_sum.bins_subsat[0],var_sum.bins_subsat[-1]+dbl,dbl))
    samples_interp = samples.interp(bins_cape=np.arange(var_sum.bins_cape[0],var_sum.bins_cape[-1]+dbl,dbl),
                          bins_subsat=np.arange(var_sum.bins_subsat[0],var_sum.bins_subsat[-1]+dbl,dbl))
    
    cp = ax.pcolormesh(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, -86400*var_mean_interp
                       , vmin=-50, vmax=50, cmap='RdBu', zorder=2)
    #ax.contour(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, var_mean_interp, levels=5,colors=['k'],
    #          linewidths=0.5,zorder=3)
    cbar = plt.colorbar(cp, ax=ax, shrink=0.7)
    cbar.set_label('(kg m$^{-2}$ s$^{-1}$)',fontsize=10)
    
    # add pdf of samples 
    ax.contour(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, samples_interp/samples_interp.sum()
               , levels=[0.01,0.02,0.03,0.04,0.05], colors=['grey'],linewidths=1,zorder=3)    

    # referecne line of BL = 0K
    ax.plot(np.linspace(-2,10,13),np.linspace(-2,10,13), color='k', lw=1)
    
    ax.set_xlabel('B$_{L,subsat}$ (K)',fontsize=10)
    ax.set_ylabel('B$_{L,cape}$ (K)',fontsize=10)
    ax.set_title(mcs_phase, fontsize=11)
    ax.set_ylim([-2,10])
    ax.set_xlim([-2,12])
    ax.set_yticks([0,2,4,6,8,10])
    ax.set_xticks([0,2,4,6,8,10])
    ax.grid(linestyle=':',lw=1,color='grey',zorder=1)
    
plt.suptitle('Variable: {}'.format(var_name),fontsize=10, fontweight='bold',y=1)
plt.tight_layout()

In [ ]:
%%time
ds_ISHF = BL2dhist_var_condition(var_name='ISHF', year_list=np.arange(2001,2011))

In [ ]:
fig,((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3,figsize=(10,5))

var_name = 'ISHF'

for ax,mcs_phase in zip([ax1,ax2,ax3,ax4,ax5,ax6],['CCS','Init','Grow','Mature','Decay','End']):
    
    samples = ds_ISHF.sel(mcs_phase=mcs_phase).samples
    var_sum = ds_ISHF.sel(mcs_phase=mcs_phase).var_sum
    var_num = ds_ISHF.sel(mcs_phase=mcs_phase).var_num
    
    # smoothening samples
    var_mean = var_sum/var_num
    dbl = 1
    var_mean_interp = var_mean.interp(bins_cape=np.arange(var_sum.bins_cape[0],var_sum.bins_cape[-1]+dbl,dbl),
                          bins_subsat=np.arange(var_sum.bins_subsat[0],var_sum.bins_subsat[-1]+dbl,dbl))
    samples_interp = samples.interp(bins_cape=np.arange(var_sum.bins_cape[0],var_sum.bins_cape[-1]+dbl,dbl),
                          bins_subsat=np.arange(var_sum.bins_subsat[0],var_sum.bins_subsat[-1]+dbl,dbl))
    
    cp = ax.pcolormesh(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, -86400*var_mean_interp
                       , vmin=-50, vmax=50, cmap='RdBu', zorder=2)
    #ax.contour(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, var_mean_interp, levels=5,colors=['k'],
    #          linewidths=0.5,zorder=3)
    cbar = plt.colorbar(cp, ax=ax, shrink=0.7)
    cbar.set_label('(kg m$^{-2}$ s$^{-1}$)',fontsize=10)
    
    # add pdf of samples 
    ax.contour(var_mean_interp.bins_subsat, var_mean_interp.bins_cape, samples_interp/samples_interp.sum()
               , levels=[0.01,0.02,0.03,0.04,0.05], colors=['grey'],linewidths=1,zorder=3)    

    # referecne line of BL = 0K
    ax.plot(np.linspace(-2,10,13),np.linspace(-2,10,13), color='k', lw=1)
    
    ax.set_xlabel('B$_{L,subsat}$ (K)',fontsize=10)
    ax.set_ylabel('B$_{L,cape}$ (K)',fontsize=10)
    ax.set_title(mcs_phase, fontsize=11)
    ax.set_ylim([-2,10])
    ax.set_xlim([-2,12])
    ax.set_yticks([0,2,4,6,8,10])
    ax.set_xticks([0,2,4,6,8,10])
    ax.grid(linestyle=':',lw=1,color='grey',zorder=1)
    
plt.suptitle('Variable: {}'.format(var_name),fontsize=10, fontweight='bold',y=1)
plt.tight_layout()